In [ ]:
import ipywidgets as widgets
from IPython.display import display
import threading
import pyaudio
from meeting_summarizer import MeetingSummarizer

summarizer = MeetingSummarizer()

# =============== Mic Selectgion ===============

def choose_mic():
    p = pyaudio.PyAudio()
    mic_options = []

    for i in range(p.get_device_count()):
        mic_name = p.get_device_info_by_host_api_device_index(0, i).get('name')
        mic_options.append((mic_name, i))

    p.terminate()
    return mic_options

mic_dropdown = widgets.Dropdown(
    options=choose_mic(),
    description='Mic:',
    disabled=False,
)
display(mic_dropdown)

# =============== Buttons/Indicators ===============

summary_output = widgets.Output(
    layout={'border': '1px solid gray',
            'width': '100%',
            'height': '300px',
            'overflow_y': 'auto',
            'white_space': 'pre-wrap',  # enables wrapping
            'word_break': 'break-word'
            }
)

def summary_click(b):
    summary_indicator.layout.display = 'block'
    with summary_output:
        summary_output.clear_output() 
        summarizer.display_summary()
        
    display(summary_output)

# Buttons have functionality; indicators display text

# =============== Start Recording ===============

record_button = widgets.Button(
    description='Record', disabled=False, button_style='success', icon='microphone')

start_indicator = widgets.Text(value='Recording started...', disabled=False)

def start_click(b):
    mic_index = mic_dropdown.value
    display(start_indicator)
    def run_start():
        summarizer.start_recording(mic_index)

    record_thread = threading.Thread(target=run_start)
    record_thread.start()

    poll_transcript_queue(summarizer.recorder, summarizer)

# =============== Stop Recording ===============

stop_button = widgets.Button(
    description='Stop', disabled=False, button_style='warning', icon='stop')

stop_indicator = widgets.Text(value='Recording stopped.', disabled=False)

def stop_click(b):
    display(stop_indicator)
    summarizer.end_recording()

# =============== Summarize ===============

summary_button = widgets.Button(
    description='Summarize', disabled=False, button_style='info')

summary_indicator = widgets.Text(value='Summary started...', disabled=False)
summary_output = widgets.Output()

def summary_click(b):
    summary_indicator.layout.display = 'block'
    with summary_output:
        summary_output.clear_output() 
        summarizer.display_summary()
        
    display(summary_output)

# =============== Ask Question (User Input) ===============

question_indicator = widgets.Text(
    value='',
    placeholder='[Press Enter To Submit]',
    description='Ask a Question: ',
    disabled=False,
    layout=widgets.Layout(width='600px')
)

question_output = widgets.Output()

def ask_click(b, width=100):
    question = question_indicator.value.strip()
    if not question:
        print("Failed To Recieve Question.")
    answer = summarizer.ask(question)

    # print('Answering...')
    with question_output:
        question_output.clear_output()
        print(f'You Asked: {question}')

        print(answer)

# =============== Live Output ===============
live_output_box = widgets.Textarea(
    value='',
    description='Live Transcript',
    layout=widgets.Layout(width='100%', height='200px')
)

def poll_transcript_queue(recorder, summarizer):
    if not summarizer.poll_timer:
        summarizer.poll_timer == None

    def poll():
        while not recorder.queue.empty():
            new_text = recorder.queue.get()
            live_output_box.value += new_text.strip('"text"') + '\n'
        if (recorder.is_recording()):
            # Calls poll() every 0.5 sec, writes new text every 0.5 sec
            timer = threading.Timer(0.5, poll)
            timer.start()

    poll()

# =============== Display/Function Calls ===============

display(record_button, stop_button, summary_button)
display(live_output_box)

record_button.on_click(start_click)
stop_button.on_click(stop_click)
question_indicator.on_submit(ask_click)
display(question_indicator)
display(question_output)
# question_button.on_click(question_click)
# submit_button.on_click(ask_click)
summary_button.on_click(summary_click)

Dropdown(description='Mic:', options=(('Brandon’s iPhone (2) Microphone', 0), ('MacBook Pro Microphone', 1), (…

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle())

Button(button_style='info', description='Summarize', style=ButtonStyle())

Textarea(value='', description='Live Transcript', layout=Layout(height='200px', width='100%'))

Text(value='', description='Ask a Question: ', layout=Layout(width='600px'), placeholder='[Press Enter To Subm…

Output()

Text(value='Recording started...')

Text(value='Recording stopped.')

Stopped recording.
